# Cutting Videos (all Files)
The Goal of this Script, is cutting both perspectives of every Videopair at the exact same moment (Frame). for this we iterate through all video files, edit, create new files with other formats that can ultimately be used to read out the data in the next step.

## Convert Videofiles
convert all Videofiles (.MOV) into (.Mp4)
The os-package enables an automation-filemanagement for this task 

In [2]:
import os
files = os.listdir(os.getcwd()+"/videos/")

for i, file in enumerate(files):
    try:
        print(file)
        pre, ext = os.path.splitext(file)
        os.rename(os.getcwd()+"/videos/"+file, os.getcwd()+"/videos/"+pre+".mp4")

        print(f'success: {i}/{len(files)}')
    except Exception as e:
        print("except", i)
        print(e)

Gehen-1.MOV
success: 0/3
Gehen-2.MOV
success: 1/3
Laufen-9_hinten.mp4
success: 2/3


## Create Audiofiles
Extracting the audios (.MAV) from the videofiles  and save them into an own directory

In [ ]:
import moviepy.editor as mp

videoFilesPath = os.getcwd()+"/videos/"
videoFiles = os.listdir(videoFilesPath)
audioFilesPath = os.getcwd()+"/audios/"

for file in videoFiles:
    pre, ext = os.path.splitext(file)
    #print(file[-15:-4])
    #if pre in ["Gehen-2"]:  
    if not file[-15:-4] == "geschnitten":
        print(file)
        try:
            my_clip = mp.VideoFileClip(videoFilesPath+file)
            audio_frame = my_clip.audio.to_soundarray()
            my_clip.audio.write_audiofile(audioFilesPath+pre+".wav")
            
        except Exception as e:
            print(e)

chunk:  93%|█████████▎| 1417/1519 [00:20<00:00, 823.46it/s, now=None]

## Cut the Videofiles

By anlyzing the soundarray of each videofile, we were able to detect the loudest marker, which was the clap we set on purpose to syncronsize the videos. The Goal is to map both perspectives of the same runner for the same paste. 
Again we're handling this with automation by writing a python script.  

In [9]:
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
from pydub.playback import play
from datetime import datetime
from multiprocessing import Process
from moviepy.editor import *

videoFilesPath = os.getcwd()+"/videos/"
videoFiles = os.listdir(videoFilesPath)
audioFilesPath = os.getcwd()+"/audios/"
audioFiles = os.listdir(audioFilesPath)

for i, fileName in enumerate(audioFiles):
    preAudio, ext = os.path.splitext(fileName)
    # if preAudio in ["Gehen-2"]:
    print(preAudio)
    if not preAudio[-11:] == "geschnitten":

        print("AUDIO :", preAudio)
        file = audioFilesPath+fileName
        try:
            # Retrieve the data from the wav file
            data, samplerate = sf.read(file)

            n = len(data)  # the length of the arrays contained in data
            Fs = samplerate  # the sample rate

            # Working with stereo audio, there are two channels in the audio data.
            # Let's retrieve each channel seperately:
            ch1 = np.array([data[i][0] for i in range(n)])  # channel 1
            ch2 = np.array([data[i][1] for i in range(n)])  # channel 2
            # x-axis and y-axis to plot the audio data

            time_axis = np.linspace(0, n / Fs, n, endpoint=False)
            sound_axis = ch1 #we only focus on the first channel here

            max_value = -9999999
            i = 0
            max_pos = 0
            for sound in sound_axis:
                if sound>max_value:
                    max_value = sound
                    max_pos= i+1
                i += 1

            seconds_start = max_pos/44100

            for fileName_v in videoFiles:
                preVideo, ext = os.path.splitext(fileName_v)
                if preAudio == preVideo:
                    file = videoFilesPath+fileName_v
                    try:
                        # loading video gfg
                        clip = VideoFileClip(file) # resize (keep aspect ratio))
                        print(clip.rotation)
                        if clip.rotation == 90:
                            clip = clip.resize(clip.size[::-1])
                            clip.rotation = 0

                        w1 = clip.w
                        h1 = clip.h
                        print("Width x Height of clip 1 : ", end = " ")
                        print(str(w1) + " x ", str(h1))

                        # getting only first 5 seconds
                        clip = clip.subclip(seconds_start, 60+seconds_start)

                        w1 = clip.w
                        h1 = clip.h
                        clip.write_videofile(file.split(".")[0]+"_geschnitten.mp4")

                    except Exception as e:
                        print(e)
            print(f'{i} /{len(audioFiles)}')
            
        except Exception as e:
            print(e)

Gehen-2
AUDIO : Gehen-2
2.2606575963718822
90
Width x Height of clip 1 :  1080 x  1920
Moviepy - Building video c:\Users\robin\WI07_Repos\running_pose_analytics/videos/Gehen-2_geschnitten.mp4.
MoviePy - Writing audio in Gehen-2_geschnittenTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video c:\Users\robin\WI07_Repos\running_pose_analytics/videos/Gehen-2_geschnitten.mp4



Moviepy - Done !
Moviepy - video ready c:\Users\robin\WI07_Repos\running_pose_analytics/videos/Gehen-2_geschnitten.mp4
3132423 /60
